In [ ]:
# !pip install pandas-ta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta


In [ ]:
data = pd.read_csv("../data/22Y/22Y_tatasteel_1D.csv",index_col="date")
data

In [ ]:
# Adding indicators
data['RSI']=ta.rsi(data.close, length=15)
data['EMA_20']=ta.ema(data.close, length=20)
data['EMA_100']=ta.ema(data.close, length=100)
data['EMA_150']=ta.ema(data.close, length=150)
data['EMA_200']=ta.ema(data.close, length=200)
data['EMA_500']=ta.ema(data.close, length=500)


data['target'] = data['close']-data.open
data['target'] = data['target'].shift(-1)

data['trend'] = [1 if data.target[i]>0 else 0 for i in range(len(data))]

# Dropping null values after adding indicators and features
data.dropna(inplace=True)
data.reset_index(inplace = True)
# data.drop(['volume', 'date'], axis=1, inplace=True) # not required
data.drop([ 'date','target'], axis=1, inplace=True) # not required

In [ ]:
data.shape

In [ ]:
# Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data)
print(data_set_scaled)

In [ ]:
# creating X,Y sets
X = []
backcandles = 30
for j in range(data_set_scaled[0].size-1): # for each col
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]): # for each data set, store curr col value of last 30 (backcandles) days
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])
print(X.shape)

# creating y and converting x to np array
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
# rows from 30th position to end cuz, we can't create training set for first 30 days, min 30 day required
# -1 because we are predicting trend which is last in cols

y=np.reshape(yi,(len(yi),1)) 
# converting 1d array to 2d ([1,2,3,4]--> [[1],[2],[3],[4]])
#  
print(X.shape)
print(y.shape)

In [ ]:
# splitting data
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
# !pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed

from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)


In [ ]:
## THE OG
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2]), name='lstm_input'))

model.add(LSTM(150, name='first_layer',activation="relu"))
model.add(Dense(1, name='dense_layer',activation="sigmoid"))

model.compile( loss = 'mean_squared_error', optimizer=optimizers.Adam())
model.fit(x=X_train, y=y_train)

In [ ]:
y_pred = model.predict(X_test)
X_pred = model.predict(X_train)
# input size = X,30,12
# where, X = number of predictig examples
# 30 = look back days which is 30 in our case
# 12 = number of featues

# so we will be proving all 12 features of previuos 30 days. and each 30 day will be considered as 1 set


In [ ]:
y_pred.shape

In [ ]:
def classify(pred):
    for i in range(len(pred)):
        if pred[i]>=0.5:
            pred[i]=1
        else:
            pred[i]=0
classify(y_pred)
classify(X_pred)

In [ ]:
from sklearn.metrics import accuracy_score
print("on training data - ",accuracy_score(X_pred,y_train))
print("on test data - ",accuracy_score(y_pred,y_test))
# increasing number of layers did increased the accurarcy to 0.54

In [ ]:
# model.save('../models/lstm_trend_1.0')